In [1]:
# Etapa 1: Upload do vídeo
from google.colab import files
import os

uploaded = files.upload()

# Pega o nome do vídeo enviado
video_filename = list(uploaded.keys())[0]
print(f"Vídeo recebido: {video_filename}")


Saving WhatsApp Video 2025-05-04 at 12.08.15.mp4 to WhatsApp Video 2025-05-04 at 12.08.15.mp4
Vídeo recebido: WhatsApp Video 2025-05-04 at 12.08.15.mp4


In [12]:
# Etapa 2: Instala o OpenCV (caso ainda não esteja instalado)
!pip install opencv-python-headless

In [13]:
# Etapa 3: Função para extrair 1 frame por segundo
import cv2
import os
import time
import psutil

def extrair_frames_por_segundo(video_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Erro ao abrir o vídeo: {video_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)  # Frames por segundo do vídeo
    frame_interval = int(fps)        # Pular para o próximo segundo

    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Salva somente 1 frame por segundo
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{saved_count:05d}.png")
            cv2.imwrite(frame_filename, frame)
            print(f"Salvando frame do segundo {saved_count}: {frame_filename}")
            saved_count += 1

        frame_count += 1

    cap.release()
    print(f"{saved_count} frames salvos (1 por segundo) em: '{output_folder}'")
    return saved_count

In [14]:
# Etapa 4: Função para gerar KPIs
def gerar_kpis(video_path, pasta_frames, tempo_inicio, tempo_fim, frames_extraidos):
    # Calcula o tempo de processamento
    tempo_total = tempo_fim - tempo_inicio

    # Uso de memória (em MB)
    processo = psutil.Process(os.getpid())
    memoria_usada = processo.memory_info().rss / (1024 * 1024)

    # Abre o vídeo para pegar o total de frames
    cap = cv2.VideoCapture(video_path)
    fps_video = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()

    # Calcula o tamanho total dos frames extraídos
    arquivos = os.listdir(pasta_frames)
    total_tamanho = sum(os.path.getsize(os.path.join(pasta_frames, f)) for f in arquivos) / (1024 * 1024)  # em MB

    # Tamanho médio por frame (em KB)
    tamanho_medio_frame = (total_tamanho * 1024) / len(arquivos)  # em KB

    # Eficiência do processamento
    eficiencia = (frames_extraidos / total_frames) * 100

    # Velocidade de extração (frames por segundo)
    velocidade_extracao = frames_extraidos / tempo_total if tempo_total > 0 else 0

    # Exibindo os KPIs
    print("\nKPIs do Processamento de Vídeo:")
    print(f"⏱️ Tempo de processamento total: {tempo_total:.2f} segundos")
    print(f"🧠 Uso de memória RAM: {memoria_usada:.2f} MB")
    print(f"💾 Espaço total dos frames extraídos: {total_tamanho:.2f} MB")
    print(f"📁 Tamanho médio por frame: {tamanho_medio_frame:.2f} KB")
    print(f"🧩 Eficiência do processamento: {eficiencia:.2f}%")
    print(f"⚙️ Velocidade de extração: {velocidade_extracao:.2f} frames/s")


# Etapa 5: Executar o processo
output_folder = "frames_1fps"
tempo_inicio = time.time()
frames_extraidos = extrair_frames_por_segundo(video_filename, output_folder)
tempo_fim = time.time()

# Gerar KPIs após a extração dos frames
gerar_kpis(video_filename, output_folder, tempo_inicio, tempo_fim, frames_extraidos)


Salvando frame do segundo 0: frames_1fps/frame_00000.png
Salvando frame do segundo 1: frames_1fps/frame_00001.png
Salvando frame do segundo 2: frames_1fps/frame_00002.png
Salvando frame do segundo 3: frames_1fps/frame_00003.png
Salvando frame do segundo 4: frames_1fps/frame_00004.png
Salvando frame do segundo 5: frames_1fps/frame_00005.png
Salvando frame do segundo 6: frames_1fps/frame_00006.png
Salvando frame do segundo 7: frames_1fps/frame_00007.png
8 frames salvos (1 por segundo) em: 'frames_1fps'

KPIs do Processamento de Vídeo:
⏱️ Tempo de processamento total: 1.47 segundos
🧠 Uso de memória RAM: 144.07 MB
💾 Espaço total dos frames extraídos: 3.48 MB
📁 Tamanho médio por frame: 445.40 KB
🧩 Eficiência do processamento: 3.46%
⚙️ Velocidade de extração: 5.43 frames/s


In [11]:
# Etapa 6: Compactar os frames para facilitar download
import shutil

shutil.make_archive("frames_extraidos", 'zip', output_folder)
files.download("frames_extraidos.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>